IMPORTS

In [26]:
conda install -c conda-forge jupyterlab

^C

Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import seaborn as sns
from ast import literal_eval
import json
%matplotlib inline

LOADING THE DATASET

In [2]:
# Visualitzarem només 3 decimals per mostra
pd.set_option('display.float_format', lambda x: '%.3f' % x)

# Funcio per a llegir dades en format csv
def load_dataset(path):
    df = pd.read_csv(path, header=0, delimiter=',')
    return df

# Carreguem dataset d'exemple
movie_df = load_dataset('tmdb_5000_movies.csv')
credits_df = load_dataset('tmdb_5000_credits.csv')
df = pd.merge(movie_df, credits_df, left_on = 'id', right_on = 'movie_id') #Left merge by id
data = df.values

x = data[:, :2]
y = data[:, 2]

#print("Dimensionalitat de la BBDD:", dataset.shape)
#print("Dimensionalitat de les entrades X", x.shape)
#print("Dimensionalitat de l'atribut Y", y.shape)

DATACLEARING AND FORMATTING

In [3]:
df.head(5)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,spoken_languages,status,tagline,title_x,vote_average,vote_count,movie_id,title_y,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.438,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.200,11800,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.083,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",...,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.900,4500,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.377,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...",...,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.300,4466,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.313,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...",...,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.600,9106,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.927,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]",...,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.100,2124,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [4]:
def process_df(df):
    df['release_date'] = pd.to_datetime(df['release_date'])
    json_cols = ['genres', 'keywords', 'production_countries', \
                 'production_companies', 'spoken_languages', 'cast', 'crew']
    for col in json_cols:
        df[col] = df[col].apply(json.loads) #Load the JSON columns
    return df
final_movies_df = process_df(df)

COLUMNS TO BE DROPPED
original_title: since title column is also included and original_title column has non-ASCII characters, it can be dropped.
homepage: there will be no analysis depending on the homepage of the movie, this column is uselesss for this specific analysis
overview & tagline: no text analysis will be made in this notebook. For retrieving the most important words, keywords.csv can be used
spoken_languages: original_language is included, no need
keywords:

In [5]:
del final_movies_df['movie_id'], final_movies_df['id'] #No need for the id of the movies once the tables are joined

del final_movies_df['title_x'] #not needed (rendundancy of two titles because of merge)

del final_movies_df["tagline"]  #it doesn't really hold any information

final_movies_df.rename(columns={'title_y':'title'}, inplace=True) #rename col

final_movies_df = final_movies_df[final_movies_df['status'] == 'Released'] #keep only released movies

del final_movies_df['status'], final_movies_df['original_title'] #no need for these columns anymore

In [6]:
final_movies_df.head(5)

,budget,genres,homepage,keywords,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,vote_average,vote_count,title,cast,crew
0,237000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",http://www.avatarmovie.com/,"[{'id': 1463, 'name': 'culture clash'}, {'id':...",en,"In the 22nd century, a paraplegic Marine is di...",150.438,"[{'name': 'Ingenious Film Partners', 'id': 289...","[{'iso_3166_1': 'US', 'name': 'United States o...",2009-12-10,2787965087,162.000,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",7.200,11800,Avatar,"[{'cast_id': 242, 'character': 'Jake Sully', '...","[{'credit_id': '52fe48009251416c750aca23', 'de..."
1,300000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",http://disney.go.com/disneypictures/pirates/,"[{'id': 270, 'name': 'ocean'}, {'id': 726, 'na...",en,"Captain Barbossa, long believed to be dead, ha...",139.083,"[{'name': 'Walt Disney Pictures', 'id': 2}, {'...","[{'iso_3166_1': 'US', 'name': 'United States o...",2007-05-19,961000000,169.000,"[{'iso_639_1': 'en', 'name': 'English'}]",6.900,4500,Pirates of the Caribbean: At World's End,"[{'cast_id': 4, 'character': 'Captain Jack Spa...","[{'credit_id': '52fe4232c3a36847f800b579', 'de..."
2,245000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",http://www.sonypictures.com/movies/spectre/,"[{'id': 470, 'name': 'spy'}, {'id': 818, 'name...",en,A cryptic message from Bond’s past sends him o...,107.377,"[{'name': 'Columbia Pictures', 'id': 5}, {'nam...","[{'iso_3166_1': 'GB', 'name': 'United Kingdom'...",2015-10-26,880674609,148.000,"[{'iso_639_1': 'fr', 'name': 'Français'}, {'is...",6.300,4466,Spectre,"[{'cast_id': 1, 'character': 'James Bond', 'cr...","[{'credit_id': '54805967c3a36829b5002c41', 'de..."
3,250000000,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...",http://www.thedarkknightrises.com/,"[{'id': 849, 'name': 'dc comics'}, {'id': 853,...",en,Following the death of District Attorney Harve...,112.313,"[{'name': 'Legendary Pictures', 'id': 923}, {'...","[{'iso_3166_1': 'US', 'name': 'United States o...",2012-07-16,1084939099,165.000,"[{'iso_639_1': 'en', 'name': 'English'}]",7.600,9106,The Dark Knight Rises,"[{'cast_id': 2, 'character': 'Bruce Wayne / Ba...","[{'credit_id': '52fe4781c3a36847f81398c3', 'de..."
4,260000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",http://movies.disney.com/john-carter,"[{'id': 818, 'name': 'based on novel'}, {'id':...",en,"John Carter is a war-weary, former military ca...",43.927,"[{'name': 'Walt Disney Pictures', 'id': 2}]","[{'iso_3166_1': 'US', 'name': 'United States o...",2012-03-07,284139100,132.000,"[{'iso_639_1': 'en', 'name': 'English'}]",6.100,2124,John Carter,"[{'cast_id': 5, 'character': 'John Carter', 'c...","[{'credit_id': '52fe479ac3a36847f813eaa3', 'de..."


One of the variables we'll be studying, later on, is the effect of having a homepage on the revenue of a movie. Given that we have a column that contains the homepage URL(link) of the movie's homepage, we'll be setting it in a binary manner. If there is a link then the movie made use of a homepage, if it doesn't then there was no website advertisement.

In [7]:
final_movies_df['has_homepage'] = 0
final_movies_df.loc[final_movies_df['homepage'].isnull() == False, 'has_homepage'] = 1
final_movies_df = final_movies_df.drop(['homepage'], axis = 1) #no need for the link anymore

C:\Users\andre\AppData\Local\Temp\ipykernel_3036\617255884.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_movies_df['has_homepage'] = 0


As stated in the description, some movies have unknown budgets and/or revenues which are set to 0 in the dataset. For our purposes, we can be content with setting the mean value of the columns to the undefined values. 

In [8]:
final_movies_df.loc[final_movies_df["budget"] == 0.0,  "budget"] = final_movies_df["budget"].mean() #mean for null values
final_movies_df.loc[final_movies_df["runtime"] == 0.0, "runtime"] = final_movies_df["runtime"].mean() #mean for null values

Next, we extract some new variables related to the release date. We'll be using the datetime(dt) functions to extract the year, month, day, and day of week information related to the release date. 
We'll also be creating the profit variable to study the monetary success of movies.

In [9]:
final_movies_df["year"] = final_movies_df["release_date"].dt.year #get year
final_movies_df["month"] = final_movies_df["release_date"].dt.month #get month 
final_movies_df["day"] = final_movies_df["release_date"].dt.day #get day
final_movies_df["profit"] = final_movies_df["revenue"] - final_movies_df["budget"] #calculate profit

Just like music, movies are also viewed based on the decade they released in. "Oh the 90s movies were so much better than 2000s and 2010s". We'll see how that statement holds on later on ;). We'll be creating a transformation function that takes the year as input and changes it to a decade (by removing the last digit of the year and changing it with a 0) then we apply it to the release year to create our "decade" column.

In [10]:
def extract_decade(x):
    return str(math.floor(x/10)*10)+"s" #Change the last digit with 0 and add "s" => 2015 => "2010s" 
    
final_movies_df= final_movies_df.fillna(0)
final_movies_df["decade"] = final_movies_df["year"].apply(extract_decade)
final_movies_df = final_movies_df[(final_movies_df['decade'] != '1910s') & (final_movies_df['decade'] != '0s')] #outliers
final_movies_df = final_movies_df.sort_values(by=['decade'], ascending=True)

In [11]:
final_movies_df.head(5)

,budget,genres,keywords,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,...,vote_count,title,cast,crew,has_homepage,year,month,day,profit,decade
4594,379000.000,"[{'id': 18, 'name': 'Drama'}, {'id': 10402, 'n...","[{'id': 4344, 'name': 'musical'}, {'id': 10229...",en,"Harriet and Queenie Mahoney, a vaudeville act,...",0.969,"[{'name': 'Metro-Goldwyn-Mayer (MGM)', 'id': 8...","[{'iso_3166_1': 'US', 'name': 'United States o...",1929-02-08 00:00:00,4358000,...,19,The Broadway Melody,"[{'cast_id': 8, 'character': 'Eddie Kearns', '...","[{'credit_id': '5647d9b9c3a368260100563e', 'de...",0,1929.000,2.000,8.000,3979000.000,1920s
4457,29092673.727,"[{'id': 18, 'name': 'Drama'}, {'id': 53, 'name...","[{'id': 212, 'name': 'london england'}, {'id':...",de,The rise and inevitable fall of an amoral but ...,1.824,"[{'name': 'Nero Films', 'id': 4903}]","[{'iso_3166_1': 'DE', 'name': 'Germany'}]",1929-01-30 00:00:00,0,...,45,Pandora's Box,"[{'cast_id': 7, 'character': 'Lulu', 'credit_i...","[{'credit_id': '52fe428ac3a36847f8026e81', 'de...",0,1929.000,1.000,30.000,-29092673.727,1920s
4661,245000.000,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...","[{'id': 2504, 'name': 'world war i'}, {'id': 1...",en,The story of an idle rich boy who joins the US...,0.786,"[{'name': 'Metro-Goldwyn-Mayer (MGM)', 'id': 8...","[{'iso_3166_1': 'US', 'name': 'United States o...",1925-11-05 00:00:00,22000000,...,21,The Big Parade,"[{'cast_id': 23, 'character': 'James Apperson'...","[{'credit_id': '52fe4381c3a36847f805922b', 'de...",0,1925.000,11.000,5.000,21755000.000,1920s
2638,92620000.000,"[{'id': 18, 'name': 'Drama'}, {'id': 878, 'nam...","[{'id': 312, 'name': 'man vs machine'}, {'id':...",de,In a futuristic city sharply divided between t...,32.352,"[{'name': 'Paramount Pictures', 'id': 4}, {'na...","[{'iso_3166_1': 'DE', 'name': 'Germany'}]",1927-01-10 00:00:00,650422,...,657,Metropolis,"[{'cast_id': 10, 'character': 'Maria', 'credit...","[{'credit_id': '52fe420fc3a36847f8000c55', 'de...",0,1927.000,1.000,10.000,-91969578.000,1920s
4200,1644736.000,"[{'id': 35, 'name': 'Comedy'}, {'id': 10749, '...","[{'id': 2693, 'name': 'monopoly'}, {'id': 5728...",en,"Alice, the only relatively normal member of th...",3.426,"[{'name': 'Columbia Pictures', 'id': 5}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1938-08-23 00:00:00,7433101,...,79,You Can't Take It With You,"[{'cast_id': 2, 'character': 'Alice Sycamore',...","[{'credit_id': '52fe45539251416c9102e8cd', 'de...",0,1938.000,8.000,23.000,5788365.000,1930s


Runtime statistics of movies: One of the metrics that we can look at is the description runtime of movies. As that statement just mentioned, Pandas offers a function that describes a column via some key statistics.

We can achieve this task with the following code:

In [12]:
print('The run time statistics are as follows')
print(final_movies_df["runtime"].describe()[['mean','std','25%','50%','75%','max']])
max_runtime_id = final_movies_df["runtime"].idxmax()
print(final_movies_df.loc[max_runtime_id][['title','runtime','year']])

The run time statistics are as follows
mean   107.610
std     20.758
25%     94.000
50%    104.000
75%    118.000
max    338.000
Name: runtime, dtype: float64
title       Carlos
runtime    338.000
year      2010.000
Name: 2384, dtype: object


print("Per comptar el nombre de valors no existents:")
print(dataset.isnull().sum())

print("Per veure estadístiques dels atributs numèrics de la BBDD:")
dataset.describe()

In [13]:
final_movies_df.columns

Index(['budget', 'genres', 'keywords', 'original_language', 'overview',
       'popularity', 'production_companies', 'production_countries',
       'release_date', 'revenue', 'runtime', 'spoken_languages',
       'vote_average', 'vote_count', 'title', 'cast', 'crew', 'has_homepage',
       'year', 'month', 'day', 'profit', 'decade'],
      dtype='object')

In [14]:
final_movies_df.columns

Index(['budget', 'genres', 'keywords', 'original_language', 'overview',
       'popularity', 'production_companies', 'production_countries',
       'release_date', 'revenue', 'runtime', 'spoken_languages',
       'vote_average', 'vote_count', 'title', 'cast', 'crew', 'has_homepage',
       'year', 'month', 'day', 'profit', 'decade'],
      dtype='object')

In [15]:
#observem que les següents columnes contenen llistes de diccionaris, amb id i noms; així doncs, realment només ens interessa el nom (genere, etc..)
#per tant, primer de tot transformem les columnes a llistes de generes, productors,...

In [16]:
drop_df = ["overview", "spoken_languages", "keywords"]
df_m = final_movies_df.drop(drop_df, axis=1) # drops the selected columns
df_m.dropna(how="all",inplace=True) # if each column is NaN or null in a row, drops this row

In [17]:
df_m.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4793 entries, 4594 to 986
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4793 non-null   float64
 1   genres                4793 non-null   object 
 2   original_language     4793 non-null   object 
 3   popularity            4793 non-null   float64
 4   production_companies  4793 non-null   object 
 5   production_countries  4793 non-null   object 
 6   release_date          4793 non-null   object 
 7   revenue               4793 non-null   int64  
 8   runtime               4793 non-null   float64
 9   vote_average          4793 non-null   float64
 10  vote_count            4793 non-null   int64  
 11  title                 4793 non-null   object 
 12  cast                  4793 non-null   object 
 13  crew                  4793 non-null   object 
 14  has_homepage          4793 non-null   int64  
 15  year               

observem que el dataset, ara sembla estar "maco" --> no conté columnes amb valors nulls / vuits

In [18]:
df_m["popularity"] =pd.to_numeric(df['popularity'], errors='coerce', downcast="float") 
df_m["budget"] =pd.to_numeric(df['budget'], errors='coerce', downcast="float") 

In [19]:
df.head(5)

,budget,genres,homepage,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,vote_average,vote_count,title,cast,crew
0,237000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",http://www.avatarmovie.com/,"[{'id': 1463, 'name': 'culture clash'}, {'id':...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.438,"[{'name': 'Ingenious Film Partners', 'id': 289...","[{'iso_3166_1': 'US', 'name': 'United States o...",2009-12-10,2787965087,162.000,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,7.200,11800,Avatar,"[{'cast_id': 242, 'character': 'Jake Sully', '...","[{'credit_id': '52fe48009251416c750aca23', 'de..."
1,300000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",http://disney.go.com/disneypictures/pirates/,"[{'id': 270, 'name': 'ocean'}, {'id': 726, 'na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.083,"[{'name': 'Walt Disney Pictures', 'id': 2}, {'...","[{'iso_3166_1': 'US', 'name': 'United States o...",2007-05-19,961000000,169.000,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,6.900,4500,Pirates of the Caribbean: At World's End,"[{'cast_id': 4, 'character': 'Captain Jack Spa...","[{'credit_id': '52fe4232c3a36847f800b579', 'de..."
2,245000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",http://www.sonypictures.com/movies/spectre/,"[{'id': 470, 'name': 'spy'}, {'id': 818, 'name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.377,"[{'name': 'Columbia Pictures', 'id': 5}, {'nam...","[{'iso_3166_1': 'GB', 'name': 'United Kingdom'...",2015-10-26,880674609,148.000,"[{'iso_639_1': 'fr', 'name': 'Français'}, {'is...",Released,6.300,4466,Spectre,"[{'cast_id': 1, 'character': 'James Bond', 'cr...","[{'credit_id': '54805967c3a36829b5002c41', 'de..."
3,250000000,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...",http://www.thedarkknightrises.com/,"[{'id': 849, 'name': 'dc comics'}, {'id': 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.313,"[{'name': 'Legendary Pictures', 'id': 923}, {'...","[{'iso_3166_1': 'US', 'name': 'United States o...",2012-07-16,1084939099,165.000,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,7.600,9106,The Dark Knight Rises,"[{'cast_id': 2, 'character': 'Bruce Wayne / Ba...","[{'credit_id': '52fe4781c3a36847f81398c3', 'de..."
4,260000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",http://movies.disney.com/john-carter,"[{'id': 818, 'name': 'based on novel'}, {'id':...",en,John Carter,"John Carter is a war-weary, former military ca...",43.927,"[{'name': 'Walt Disney Pictures', 'id': 2}]","[{'iso_3166_1': 'US', 'name': 'United States o...",2012-03-07,284139100,132.000,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,6.100,2124,John Carter,"[{'cast_id': 5, 'character': 'John Carter', 'c...","[{'credit_id': '52fe479ac3a36847f813eaa3', 'de..."


In [20]:
df_m.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4793 entries, 4594 to 986
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4793 non-null   float32
 1   genres                4793 non-null   object 
 2   original_language     4793 non-null   object 
 3   popularity            4793 non-null   float32
 4   production_companies  4793 non-null   object 
 5   production_countries  4793 non-null   object 
 6   release_date          4793 non-null   object 
 7   revenue               4793 non-null   int64  
 8   runtime               4793 non-null   float64
 9   vote_average          4793 non-null   float64
 10  vote_count            4793 non-null   int64  
 11  title                 4793 non-null   object 
 12  cast                  4793 non-null   object 
 13  crew                  4793 non-null   object 
 14  has_homepage          4793 non-null   int64  
 15  year               

Since there is 1 null release_date/release_year entries and filling that is not logical, they will be dropped, too. 
And also 1 row that has null as in column release_date may be dropped.
For runtime, again a similar case occurs and it can be handled by filling NaN values with the mean.

In [21]:
df_m["runtime"].fillna(df_m["runtime"].mean(), inplace=True)

In [22]:
df_m.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4793 entries, 4594 to 986
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4793 non-null   float32
 1   genres                4793 non-null   object 
 2   original_language     4793 non-null   object 
 3   popularity            4793 non-null   float32
 4   production_companies  4793 non-null   object 
 5   production_countries  4793 non-null   object 
 6   release_date          4793 non-null   object 
 7   revenue               4793 non-null   int64  
 8   runtime               4793 non-null   float64
 9   vote_average          4793 non-null   float64
 10  vote_count            4793 non-null   int64  
 11  title                 4793 non-null   object 
 12  cast                  4793 non-null   object 
 13  crew                  4793 non-null   object 
 14  has_homepage          4793 non-null   int64  
 15  year               

In [25]:
df_m.head(5)

,budget,genres,original_language,popularity,production_companies,production_countries,release_date,revenue,runtime,vote_average,vote_count,title,cast,crew,has_homepage,year,month,day,profit,decade
4594,379000.000,"[{'id': 18, 'name': 'Drama'}, {'id': 10402, 'n...",en,0.969,"[{'name': 'Metro-Goldwyn-Mayer (MGM)', 'id': 8...","[{'iso_3166_1': 'US', 'name': 'United States o...",1929-02-08 00:00:00,4358000,100.000,5.000,19,The Broadway Melody,"[{'cast_id': 8, 'character': 'Eddie Kearns', '...","[{'credit_id': '5647d9b9c3a368260100563e', 'de...",0,1929.000,2.000,8.000,3979000.000,1920s
4457,0.000,"[{'id': 18, 'name': 'Drama'}, {'id': 53, 'name...",de,1.824,"[{'name': 'Nero Films', 'id': 4903}]","[{'iso_3166_1': 'DE', 'name': 'Germany'}]",1929-01-30 00:00:00,0,109.000,7.600,45,Pandora's Box,"[{'cast_id': 7, 'character': 'Lulu', 'credit_i...","[{'credit_id': '52fe428ac3a36847f8026e81', 'de...",0,1929.000,1.000,30.000,-29092673.727,1920s
4661,245000.000,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",en,0.786,"[{'name': 'Metro-Goldwyn-Mayer (MGM)', 'id': 8...","[{'iso_3166_1': 'US', 'name': 'United States o...",1925-11-05 00:00:00,22000000,151.000,7.000,21,The Big Parade,"[{'cast_id': 23, 'character': 'James Apperson'...","[{'credit_id': '52fe4381c3a36847f805922b', 'de...",0,1925.000,11.000,5.000,21755000.000,1920s
2638,92620000.000,"[{'id': 18, 'name': 'Drama'}, {'id': 878, 'nam...",de,32.352,"[{'name': 'Paramount Pictures', 'id': 4}, {'na...","[{'iso_3166_1': 'DE', 'name': 'Germany'}]",1927-01-10 00:00:00,650422,153.000,8.000,657,Metropolis,"[{'cast_id': 10, 'character': 'Maria', 'credit...","[{'credit_id': '52fe420fc3a36847f8000c55', 'de...",0,1927.000,1.000,10.000,-91969578.000,1920s
4200,1644736.000,"[{'id': 35, 'name': 'Comedy'}, {'id': 10749, '...",en,3.426,"[{'name': 'Columbia Pictures', 'id': 5}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1938-08-23 00:00:00,7433101,126.000,7.200,79,You Can't Take It With You,"[{'cast_id': 2, 'character': 'Alice Sycamore',...","[{'credit_id': '52fe45539251416c9102e8cd', 'de...",0,1938.000,8.000,23.000,5788365.000,1930s


AS WE CAN SEE, NOW WE HAVE OUR DATASET CORRECTLY
There are some cells, which have stringified list of json inputs such as genres, production_companies and production_countries. For easier processing, these have to be converted into list of inputs

In [23]:
# converts json list to list of inputs (from the label specified with 'wanted' parameter) --> funció trobada a internet
def json_to_arr(cell, wanted = "name"): 
    cell = literal_eval(cell)
    if cell == [] or (isinstance(cell, float) and cell.isna()):
        return np.nan
    result = []
    counter = 0
    for element in cell:
        if counter < 3:
            result.append(element[wanted])
            counter += 1
        else:
            break
    return result[:3]

In [24]:
df_m[['genres']] = df_m[['genres']].applymap(json_to_arr)
df_m[['production_countries']] = df_m[['production_countries']].applymap(lambda row: 
                                                                     json_to_arr(row, "iso_3166_1"))
df_m[['production_companies']] = df_m[['production_companies']].applymap(json_to_arr)


ValueError: malformed node or string: [{'id': 18, 'name': 'Drama'}, {'id': 10402, 'name': 'Music'}, {'id': 10749, 'name': 'Romance'}]

In [ ]:
df_m.head(10)

In [ ]:
a = 0
for i in df['revenue']:
    if i ==0:
        a=a+1
print('revenue',a)

b = 0
for i in df['budget']:
    if i ==0:
        b=b+1
print('budget',b)

Many entries of budget and revenue are 0. However, instead of 0, having NaN is more logical for seeing how many entries are actually available.

In [ ]:
df_m['budget'] = df_m['budget'].replace(0,pd.np.nan)
df_m['revenue'] = df_m['revenue'].replace(0,pd.np.nan)

In [ ]:
a = 0
for i in df_m['budget']:
    if math.isnan(i) :
        a=a+1
print(a)
print(len(df_m['budget']))
        

In [ ]:
print("Number of rows with budget < 100: ", len((df_m[(df_m["budget"].notna())&(df_m["budget"] < 100)])))
print("Number of rows with budget > 100 and < 1000: ", len(df_m[(df_m["budget"].notna())&(df_m["budget"] > 100)
                                                              &(df_m["budget"] < 1000)]))
print("Number of rows with budget > 1000 and < 10000: ", len(df_m[(df_m["budget"].notna())&(df_m["budget"] > 1000)
                                                              &(df_m["budget"] < 10000)]))

There are some rows that have a budget and revenue value, that are not actually scaled. By checking some of the notebooks shared, I have decided to move on with the scaling function below. For example, if the value is 1, then it scales to 1 million. If an example will be given from the true data:

id: 17402
Title: Miami Rhapsody
Production Company: Hollywood Pictures
Date: 1995-01-27
Budget: 6
Revenue: 5 (by looking IMDB, actual revenue can be seen as around 5 million)

In [ ]:
def scale_money(num):
    if num < 100:
        return num * 1000000
    elif num >= 100 and num < 1000:
        return num * 10000
    elif num >= 1000 and num < 10000:
        return num *100
    else:
        return num

In [ ]:
df_m[['budget', 'revenue']] = df_m[['budget', 'revenue']].applymap(scale_money)

After these steps, the columns can be osberved to see how many null or NaN entries there are. So, a heatmap and data is below:

In [ ]:
sns.heatmap(df_m.isnull(), yticklabels = False, cbar = False, cmap = 'viridis')

In [ ]:
print("NaN Genres Count: ", len(df_m[df_m["genres"].isna()]))
print("NaN Revenue Count: ", len(df_m[df_m['revenue'].isna()])) 
print("NaN Budget Count: ", len(df_m[df_m['budget'].isna()])) 
print("NaN Production Company Count: ", len(df_m[df_m["production_companies"].isna()]))
print("NaN Production Country Count: ", len(df_m[df_m["production_countries"].isna()]))

In [ ]:
plt.figure()
plt.title("Histograma de l'atribut 0")
plt.xlabel("Attribute Value")
plt.ylabel("Count")
hist = plt.hist(x[:,0], bins=11, range=[np.min(x[:,0]), np.max(x[:,0])], histtype="bar", rwidth=0.8)

In [ ]:
import seaborn as sns

# Mirem la correlació entre els atributs d'entrada per entendre millor les dades
correlacio = dataset.corr()

plt.figure()

ax = sns.heatmap(correlacio, annot=True, linewidths=.5)

In [ ]:
# Mirem la relació entre atributs utilitzant la funció pairplot
relacio = sns.pairplot(dataset)